## Flask `session`

Se hai notato, anche riavviando il server Flask, gli utenti rimangono loggati.

Come fa Flask a sapere che un utente è loggato?

Dove viene memorizzata la "session" di Flask?

Nelle applicazioni Flask, le sessioni vengono gestite attraverso un meccanismo chiamato "secure cookie session". Di default, Flask salva i dati della sessione direttamente nel cookie del browser dell'utente in forma crittografata. Questo significa che, anche se riavvii il server, le informazioni di sessione persistono finché il cookie non scade o non viene cancellato, poiché i dati necessari per autenticare la sessione sono memorizzati nel browser dell'utente e non sul server.

### Come funziona la sessione di Flask

Quando utilizzi Flask e imposti variabili nella sessione, per esempio `session['user_id'] = some_user_id`, Flask serializza queste informazioni e le salva all'interno di un cookie sul browser dell'utente. Questo cookie è crittografato usando la chiave segreta che definisci nella configurazione dell'app (`app.config['SECRET_KEY']`). Il cookie contiene quindi tutte le informazioni necessarie per ricostruire la sessione ad ogni richiesta successiva fatta dall'utente.

### Implicazioni e considerazioni di sicurezza

1. **Persistenza della sessione**: Poiché le sessioni sono memorizzate nei cookie, gli utenti rimarranno loggati anche se il server viene riavviato. Questo comportamento può essere desiderato o meno, a seconda del caso d'uso.

2. **Sicurezza della chiave Segreta**: È fondamentale che la chiave segreta (`SECRET_KEY`) sia mantenuta segreta e sia complessa abbastanza da prevenire attacchi di decifrazione del cookie.

3. **HTTPS**: È altamente raccomandato servire la tua applicazione su HTTPS per prevenire che i cookie di sessione siano intercettati durante il trasporto su reti non sicure.

### Altre opzioni per la gestione della sessione

Se preferisci non utilizzare i cookie per memorizzare le informazioni di sessione o se hai requisiti che necessitano la persistenza della sessione sul lato server, puoi considerare alternative come:

1. **Flask-Session**: Un'estensione di Flask che permette di memorizzare le sessioni sul lato server. Con Flask-Session, puoi configurare il backend della sessione per usare database come Redis, Memcached, filesystem, MongoDB, ecc.

   Per esempio, se decidessi di usare Redis come backend per le sessioni:

   ```python
   from flask_session import Session

   app.config['SESSION_TYPE'] = 'redis'
   app.config['SESSION_PERMANENT'] = False
   app.config['SESSION_USE_SIGNER'] = True
   app.config['SESSION_REDIS'] = redis.from_url('redis://localhost:6379')

   Session(app)
   ```

2. **Database Personalizzati**: Implementare una gestione della sessione personalizzata che salva e recupera dati dalla tua base dati.

L'uso di un meccanismo di sessione server-side può aiutare in scenari dove la persistenza delle sessioni è necessaria indipendentemente dallo stato del browser client o per mantenere un controllo più stretto sulla validità e sicurezza delle sessioni utente.

## Dietro le quinte: Werkzeug

In realtà, la gestione delle sessioni che abbiamo descritto è principalmente una caratteristica di [**Werkzeug**](https://pypi.org/project/Werkzeug/), che è una delle librerie di base su cui Flask è costruito.

Werkzeug fornisce molte delle funzionalità utili per il networking e la gestione delle richieste HTTP, inclusa la gestione sicura dei cookie e delle sessioni. Flask estende questa funzionalità per renderla facilmente accessibile agli sviluppatori attraverso la sua interfaccia di alto livello.

### Dettagli su Werkzeug e Flask:

1. **Werkzeug**:
   - Werkzeug gestisce la parte di crittografia e sicurezza dei cookie, e Flask sfrutta queste capacità per la sua implementazione delle sessioni.
   - La sessione in Werkzeug è rappresentata da una classe chiamata `SecureCookieSession`, che di default usa i cookie sicuri per memorizzare dati.
   - Questi cookie sono crittografati utilizzando la chiave segreta fornita (`SECRET_KEY`), e solo il server è in grado di leggere il loro contenuto grazie a questa chiave.

2. **Flask**:
   - Flask usa la classe `SecureCookieSession` di Werkzeug per implementare la sua funzione di sessione, rendendo la gestione delle sessioni semplice e diretta per gli sviluppatori che usano Flask.
   - Flask aggiunge l'interfaccia `session` che permette agli sviluppatori di memorizzare e recuperare dati dalla sessione in modo molto simile a un dizionario Python.

### Implicazioni per gli sviluppatori:

Quando utilizzi Flask e lavori con sessioni, è importante ricordare che stai interagendo con la funzionalità fornita da Werkzeug, ma attraverso le astrazioni e le interfaccie fornite da Flask. Questo significa che tutte le configurazioni e le gestioni legate alla sicurezza delle sessioni, come la definizione della `SECRET_KEY` e le opzioni di configurazione delle sessioni, sono gestite nel contesto di Flask ma operano grazie alle basi poste da Werkzeug.